In [2]:
import math
import random

In [ ]:
def ordena(populacao):
  for i in range(0, len(populacao)-1):
    for j in range(i+1, len(populacao)):
      if populacao[i][2] < populacao[j][2]:
        populacao[i], populacao[j] = populacao[j], populacao[i]
  return populacao

In [3]:
def getPeso(peso, mochila):
    pesoAtual = 0
    if len(peso) != len(mochila):
        return -1
    for i in range(0, len(mochila)):
        pesoAtual += peso[i] * int(mochila[i])
    return pesoAtual

In [4]:
def getValor(peso, valor, mochila, C):
    valorAtual = 0
    if len(peso) != len(mochila):
        return -1
    for i in range(0, len(mochila)):
        valorAtual += valor[i] * int(mochila[i])
    return valorAtual if getPeso(peso, mochila) <= C else (-1)*valorAtual

In [ ]:
def calculaFitness(populacao):
  ret = 0
  for elemento in populacao:
    ret = ret + elemento[2]
  return ret

In [ ]:
def geraPopulacaoInicial(p, v, C, tam):
  populacao = []
  for i in range(0, tam):
    estadoAtual = random.randint(0, 2 ** len(p))
    mAtual = bin(estadoAtual)[2:].zfill(len(p))
    pAtual = getPeso(p, mAtual)
    while pAtual > C:
        estadoAtual = random.randint(0, 2 ** len(p))
        mAtual = bin(estadoAtual)[2:].zfill(len(p))
        pAtual = getPeso(p, mAtual)
    vAtual = getValor(p, v, mAtual, C)
    elemento = [mAtual, pAtual, vAtual]
    populacao.append(elemento)
  populacao = ordena(populacao)
  return populacao, calculaFitness(populacao)


In [ ]:
def cruzamento(populacao, fitnessAtual, tam, p, v, C): # vetor ordenado do menor para o maior
  percTotal  = 0
  vetPerc    = []
  popNova    = []
  pontoDeCorte = random.randint(0, len(populacao[0][0]))
  for elemento in populacao:
    percTotal = percTotal + elemento[2]/fitnessAtual
    vetPerc.append(percTotal)
  for i in range(0, tam // 2):
    v = random.random()
    iv1, iv2 = 0, 0
    for j in range(0, len(vetPerc)):
      if v <= vetPerc[j]:
        iv1 = j 
        break 
    while True:
      v = random.random()
      for j in range(0, len(vetPerc)):
        if v <= vetPerc[j]:
          iv2 = j
          break
      if iv1 != iv2:
        break
    m1, m2, n = "", "", len(populacao[0][0])
    for j in range(0, n):
      if j < pontoDeCorte:
        m1 = m1 + populacao[iv1][0][j]
        m2 = m2 + populacao[iv2][0][j]
      else:
        m1 = m1 + populacao[iv2][0][j]
        m2 = m2 + populacao[iv1][0][j]
    nElemento1 = [m1, getPeso(m1, p), getValor(p, v, m1, C)]  
    nElemento2 = [m2, getPeso(m2, p), getValor(p, v, m2, C)]
    popNova.append(nElemento1)
    popNova.append(nElemento2)
  return popNova 

In [ ]:
def mutacao(populacao, percMutacao, p, v, C):
  qtd = math.ceil(percMutacao * len(populacao) / 100)
  for i in range(0, qtd):
    j = random.randint(0, len(populacao) - 1)
    k = random.randint(0, len(populacao[0][0] - 1))
    elemento = populacao[j][0]
    nElemento = ""
    for l in range(0, len(elemento)):
      if l != k:
        nElemento = nElemento + elemento[l]
      else:
        nElemento = nElemento + "0" if elemento[l] == "1" else "1"
    populacao[j] = [nElemento, getPeso(p, nElemento), getValor(p, v, nElemento, C)]
  return populacao

In [ ]:
def ag(p, v, C, tam, percMutacao):
  fitnessAtual = 0
  geracaoAtual = []
  populacao, fitnessGeracao = geraPopulacaoInicial(p, v, C, tam)
  while fitnessGeracao > fitnessAtual:
    geracaoAtual   = populacao
    fitnessAtual   = fitnessGeracao
    geracaoAtual   = cruzamento(geracaoAtual, fitnessAtual, p, v, C)
    geracaoAtual   = mutacao(geracaoAtual, percMutacao, p, v, C)
    geracaoAtual   = preSelecao(geracaoAtual)
    populacao      = selecao(populacao, geracaoAtual, tam)
    fitnessGeracao = calculaFitness(populacao)
